In [1]:
print("ok")

ok


In [1]:
%pwd

'c:\\Adani\\chat_bot\\Medical-Chat-bot-Gen-Ai\\research'

In [1]:
import os
os.chdir("../")

In [2]:
%pwd

'c:\\Adani\\chat_bot\\Medical-Chat-bot-Gen-Ai'

In [3]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [4]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [5]:
extracted_data=load_pdf_file(data='Data/')

In [8]:
# extracted_data

In [6]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [7]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 7023


In [8]:
from langchain.embeddings import HuggingFaceEmbeddings

In [9]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [10]:
embeddings = download_hugging_face_embeddings()

C:\Users\chaud\AppData\Local\Temp\ipykernel_32092\2661704553.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
c:\Users\chaud\miniconda3\envs\medbot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))


Length 384


In [31]:
from dotenv import load_dotenv
load_dotenv()

True

In [32]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
GOOGLE_API_KEY=os.environ.get('GOOGLE_API_KEY')

In [16]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot"


pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

{
    "name": "medicalbot",
    "metric": "cosine",
    "host": "medicalbot-8lxu3nj.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [33]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [18]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [19]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [20]:
docsearch

In [21]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [22]:
retrieved_docs = retriever.invoke("What is Coronary artery disease?")

In [23]:
retrieved_docs

[Document(id='aeda9833-f09f-4040-94a9-e164cf52a21a', metadata={'creationdate': '2004-12-18T17:16:32-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:35:04-06:00', 'page': 319.0, 'page_label': '320', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Gale Encyclopedia of Medicine Vol. 2 (C-F).pdf', 'total_pages': 759.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 931\nCoronary artery disease\nGEM -0625 to 1002 - C  10/22/03 6:11 PM  Page 931'),
 Document(id='2f16ae71-59f3-456b-818b-8ff7bef04b22', metadata={'creationdate': '2004-12-18T17:16:32-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:35:04-06:00', 'page': 322.0, 'page_label': '323', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Gale Encyclopedia of Medicine Vol. 2 (C-F).pdf', 'total_pages': 759.0}, page_content='disease\nCoronary heart disease see Coronary artery\ndisease\nCoronary stenting\nDefinition\nA coronary stent is an artificial support device used\nin the coronary artery to keep the vessel o

In [34]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.4,
    max_output_tokens=500,
    top_p=0.8,
    verbose=True,
    google_api_key = GOOGLE_API_KEY # Explicitly pass the API key
)

In [35]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [36]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [37]:
response = rag_chain.invoke({"input": "what is Cystitis?"})
print(response["answer"])

Cystitis is inflammation of the urinary bladder.  It is sometimes referred to as a lower urinary tract infection (UTI) when it occurs with urethritis (inflammation of the urethra).  Symptoms include pain during urination, urgency, and increased frequency of urination.


In [38]:
response = rag_chain.invoke({"input": "What is stats?"})
print(response["answer"])

I'm sorry, but this document does not explain what "stats" means in general terms.  The provided text only mentions the use of statistics in predicting disease outcomes and survival rates.
